# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

**Aleksandr Plotnikow** 



First we get raw data from wikipedia to dataframe called df

In [1]:
import sys
!{sys.executable} -m pip install geocoder

In [2]:
import pandas as pd
import numpy as np
import requests
import geocoder

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki = requests.get(url).content
df_list = pd.read_html(wiki,header=0)[0]
df = df_list
df.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We get rid of 'Not assigned' and rename columns as in the course

In [3]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
df.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"



Grouping data and reseting index 

In [4]:
df_new = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()
df_new.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df.shape

(103, 3)

In [6]:
def get_geocode(postal_code):
  
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [43]:
geo_data=pd.read_csv('http://cocl.us/Geospatial_data')

In [44]:

geo_data.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)



In [45]:
geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [46]:


df_cor = pd.merge(df, geo_data, on = "PostalCode")

df_cor.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
